In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
## Imports
import warnings
warnings.filterwarnings('ignore')

import sys

# if "google.colab" in sys.modules:
    # !sudo add-apt-repository -y ppa:deadsnakes/ppa
    # !sudo apt-get -y update
    # !sudo apt-get -y install python3.9
    # !sudo apt-get -y install python3.9-dev
    # !sudo apt-get -y install python3-pip
    # !sudo apt-get -y install python3.9-distutils
    # !python3.9 -m pip install --upgrade setuptools
    # !python3.9 -m pip install --upgrade pip
    # !python3.9 -m pip install --upgrade distlib

    # !sudo update-alternatives --set python /usr/bin/python3.9
    # !sudo ln -sf /usr/bin/python /usr/local/bin/python

    # #install python 3.9
    # !sudo apt-get update -y
    # !sudo apt-get install python3.9

    # #change alternatives
    # !sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.7 1
    # !sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.9 2

    # !python3.9 setup.py
    
    # !pip uninstall lightgbm -y
    # !pip install lightgbm==3.3.1
    # !pip uninstall catboost -y
    # !pip install catboost==1.0.6
    # !pip install Levenshtein
    # !pip uninstall pandas -y
    # !pip install pandas==1.4.1

import os
import gc
import time
import random
import pickle
# import Levenshtein
import difflib
import joblib
import multiprocessing
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
# import lightgbm as lgb
# from catboost import CatBoost
# from catboost import Pool
from tqdm.auto import tqdm
from requests import get
from collections import Counter, defaultdict
from sklearn.model_selection import GroupKFold, StratifiedKFold
from sklearn.neighbors import KNeighborsRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

In [3]:
!pip install feather-format

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import feather

In [5]:
## Parameters
class CFG:
    AUTHOR = "kuruton"
    expID = ""
    if "google.colab" in sys.modules:
        expID = get("http://172.28.0.2:9000/api/sessions").json()[0]["name"].split(".")[0].split("-")[0]
    ROOT_DIR = '/content/drive/MyDrive/Kaggle/Foursquare'
    DATASET_DIR = os.path.join(ROOT_DIR, 'Dataset')
    INPUT_DIR = os.path.join(ROOT_DIR, 'Input')
    OUTPUT_DIR = os.path.join(ROOT_DIR, 'Output')
    PROCESSED_DIR = os.path.join(ROOT_DIR, 'processed_data')
    is_debug = False
    SEED = 2022
    num_neighbors = 20
    num_split = 5
    feat_columns = ['name', 'address', 'city', 
                'state', 'zip', 'url', 
              'phone', 'categories', 'country']
    vec_columns = ['name', 'categories', 'address', 
                  'state', 'url', 'country']

def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    
seed_everything(CFG.SEED)

In [6]:
if not os.path.exists(os.path.join(CFG.OUTPUT_DIR, CFG.expID)):
    os.makedirs(os.path.join(CFG.OUTPUT_DIR, CFG.expID))

In [7]:
## Data load
if "google.colab" in sys.modules:
    data_root = CFG.INPUT_DIR
else:
    data_root = '../input/foursquare-location-matching'
test_df = pd.read_csv(os.path.join(data_root, 'train.csv'))

if CFG.is_debug:
    test_df = test_df[:10000]
    test_df = test_df.reset_index(drop = True)

In [8]:
pred_df = pd.read_csv(os.path.join(CFG.PROCESSED_DIR, "pred31_score_and_rank.csv"))
pred_df.head()

,query_ix,candidate_ix,pred,rank,fold
0,6,903995,0.015949,1.0,0
1,6,305579,0.001998,2.0,0
2,6,238967,0.264468,3.0,0
3,6,786499,0.023800,4.0,0
4,6,549946,0.026928,5.0,0


In [9]:
pred_df = pred_df[pred_df['pred'] > 0.93]

In [10]:
if CFG.is_debug:
    print(pred_df.shape)
    pred_df = pred_df[(pred_df['query_ix'] < 10000) & (pred_df['candidate_ix'] < 10000)]
    print(pred_df.shape)

In [11]:
query_col = "query_ix"
candidate_col = "candidate_ix"
score_col = "pred"

In [12]:
from itertools import groupby

class UnionFind():
    
    def __init__(self, N):
        self.parent = [-1] * N
        self.size = [1] * N
        
    def find(self, x):
        p = self.parent[x]
        if p == -1:
            return x
        p = self.find(p)
        self.parent[x] = p
        return p
    
    def unite(self, x, y):
        px = self.find(x)
        py = self.find(y)
        if px == py:
            return
        if self.size[px] < self.size[py]:
            px, py = py, px
        self.size[px] += self.size[py]
        self.parent[py] = px

In [13]:
import networkx as nx
from heapq import heappop, heappush
from tqdm.notebook import tqdm
warnings.simplefilter('ignore')

In [14]:
def get_id2poi(input_df: pd.DataFrame) -> dict:
    return dict(zip(input_df['id'], input_df['point_of_interest']))

def get_poi2ids(input_df: pd.DataFrame) -> dict:
    return input_df.groupby('point_of_interest')['id'].apply(set).to_dict()

def get_score(input_df: pd.DataFrame):
    scores = []
    for id_str, matches in zip(input_df['id'].to_numpy(), input_df['matches'].to_numpy()):
        targets = poi2ids[id2poi[id_str]]
        preds = set(matches.split())
        score = len((targets & preds)) / len((targets | preds))
        scores.append(score)
    scores = np.array(scores)
    return scores.mean()

In [15]:
id2poi = get_id2poi(test_df)
poi2ids = get_poi2ids(test_df)

In [16]:
max_dist = 2
max_N = 256
shortest_paths_threshold = 1000
eval_points = list(range(len(test_df)))

submission_df = test_df[["id"]]
submission_df["matches"] = test_df["id"]

uft = UnionFind(len(test_df))

for ix, nix in pred_df[["query_ix", "candidate_ix"]].values:
    uft.unite(ix, nix)
group_members = defaultdict(list)
group_size = defaultdict(int)
group_map = {}
for i in range(len(test_df)):
    group_members[uft.find(i)].append(i)
    group_size[uft.find(i)] = uft.size[uft.find(i)]
    group_map[i] = uft.find(i)

large_groups = set([k for k, v in group_size.items() if v > min(max_N, max_dist+1)])

pred_df["group"] = pred_df["query_ix"].map(group_map)
pred_df["size"] = pred_df["group"].map(group_size)
pred_df["left"] = np.minimum(pred_df["query_ix"], pred_df["candidate_ix"])
pred_df["right"] = np.maximum(pred_df["query_ix"], pred_df["candidate_ix"])
pred_df = pred_df.sort_values(by="pred", ascending=False).drop_duplicates(["left", "right"])

graphs = {k: nx.Graph() for k in large_groups}
for large_group in large_groups:
    for member in group_members[large_group]:
        graphs[large_group].add_node(member)

neighbors = defaultdict(list)
for l, r, s in pred_df[pred_df["group"].isin(large_groups)][["left", "right", "pred"]].values:
    l, r = int(l), int(r)
    g = uft.find(l)
    neighbors[l].append((r, s))
    neighbors[r].append((l, s))
    graphs[g].add_edge(l, r)

shortest_paths = {g: {k: d for k, d in nx.all_pairs_shortest_path_length(graphs[g])} for g in large_groups if len(group_members[g]) < shortest_paths_threshold}

matches = []
for i in eval_points:
    g = uft.find(i)
    preds = []
    if g in large_groups:
        if g in shortest_paths.keys():
            for n, d in shortest_paths[g][i].items():
                if d <= max_dist:
                    preds.append(n)
        if (len(preds) > max_N) or (not g in shortest_paths.keys()):
            searched = set()
            heapq = [(-1., 0, i)]
            while len(heapq) and (len(searched) < max_N):
                _, step, x = heappop(heapq)
                if x in searched:
                    continue
                searched.add(x)
                if step >= max_dist:
                    continue
                for n, s in neighbors[x]:
                    if n in searched:
                        continue
                    heappush(heapq, (-s, step+1, n))
            preds = list(searched)
    else:
        preds = group_members[g]
    matches.append(" ".join([test_df.at[p, "id"] for p in preds]))
submission_df["matches"] = matches

In [17]:
cv = get_score(submission_df)
print("CV:" + str(cv))

CV:0.869256548875664


In [23]:
test_df.head()

,id,name,latitude,longitude,address,city,state,zip,country,url,phone,categories,point_of_interest
0,E_000001272c6c5d,Café Stad Oudenaarde,50.859975,3.634196,Abdijstraat,Nederename,Oost-Vlaanderen,9700,BE,NaN,NaN,Bars,P_677e840bb6fc7e
1,E_000002eae2a589,Carioca Manero,-22.907225,-43.178244,NaN,NaN,NaN,NaN,BR,NaN,NaN,Brazilian Restaurants,P_d82910d8382a83
2,E_000007f24ebc95,ร้านตัดผมการาเกด,13.780813,100.484900,NaN,NaN,NaN,NaN,TH,NaN,NaN,Salons / Barbershops,P_b1066599e78477
3,E_000008a8ba4f48,Turkcell,37.844510,27.844202,Adnan Menderes Bulvarı,NaN,NaN,NaN,TR,NaN,NaN,Mobile Phone Shops,P_b2ed86905a4cd3
4,E_00001d92066153,Restaurante Casa Cofiño,43.338196,-4.326821,NaN,Caviedes,Cantabria,NaN,ES,NaN,NaN,Spanish Restaurants,P_809a884d4407fb


In [26]:
cnt = 0
for ix, nix in pred_df[["query_ix", "candidate_ix"]].values:
    cnt += test_df.loc[ix, 'point_of_interest'] == test_df.loc[nix, 'point_of_interest']
cnt / pred_df.shape[0]

0.8459506212929387

In [28]:
used = set()
for ix, nix in pred_df[["query_ix", "candidate_ix"]].values:
    used.add((ix, nix))
    used.add((nix, ix))

In [22]:
cnt = 0
for members in group_members.values():
    n = len(members)
    if n <= 2:
        continue
    

E_000001272c6c5d E_000001272c6c5d E_da7fa3963561f8


In [21]:
pred_df

,query_ix,candidate_ix,pred,rank,fold,group,size,left,right
13006080,895696,1016664,1.000000,1.0,1,895696,2,895696,1016664
36380128,1129443,153848,1.000000,1.0,4,153848,2,153848,1129443
12575040,829493,895575,1.000000,1.0,1,829493,2,829493,895575
33521920,684652,1055629,1.000000,1.0,4,684652,2,684652,1055629
31754144,407732,195082,1.000000,1.0,4,195082,2,195082,407732
...,...,...,...,...,...,...,...,...,...
32671587,552204,440692,0.930006,4.0,4,552204,2,440692,552204
15872036,203667,33503,0.930005,5.0,2,203667,2,33503,203667
19029348,696571,1024667,0.930003,5.0,2,696571,2,696571,1024667
17820770,507654,312510,0.930002,3.0,2,507654,3,312510,507654


In [29]:
submission_df['matches_num'] = submission_df['matches'].map(lambda x: len(x.split()))
submission_df['matches_num'].sum()

2593916

In [30]:
submission_df['matches_num'].sum() - pred_df.shape[0]

2095037

In [ ]:
def using_edge_betweenness_centrality(df, weight_threshold = 0.15780210284453428, remove_edge_threshold = 0.11766651703447985, graph_member_num = 3):
    G = nx.Graph()
    for i, j, w in df[['id', 'match_id', 'pred']].values:
        G.add_edge(i, j, weight=w)

    list_remove_edges = []
    list_add_edges = []
    def split_graph(G):
        list_comp = list(nx.connected_components(G))
        n = len(G.nodes)
        if len(list_comp) == 1:
            map_bet = nx.edge_betweenness_centrality(G, normalized=True)
            map_bet = {(i, j): w  for (i, j), w in map_bet.items() 
                      if G[i][j]['weight'] < weight_threshold}
            if len(map_bet) == 0:
                return
            edge, val = max(map_bet.items(), key=lambda x: x[1])
            if val > remove_edge_threshold:
                G.remove_edge(*edge)
                list_remove_edges.append(edge)
                return split_graph(G)
        else:
            iters = list_comp
            for comp in iters:
                if len(comp) > graph_member_num:
                    split_graph(nx.Graph(G.subgraph(comp)))
                    
    split_graph(G)
    for edge in list_remove_edges:
        G.remove_edge(*edge)

    def get_score(i, j):
        try:
            return G[i][j]['weight']
        except KeyError:
            return -1

    ids = df['id'].unique()
    matches = []

    for i in ids:
        if i in G:
            m = list(set([i] + list(G.neighbors(i))))
        else:
            m = [i]
        matches.append(' '.join(m))
    matched = pd.DataFrame(dict(id=ids, matches=matches))
    return matched